# Tesla Stock Price Prediction using LSTM

In this notebook, we will use LSTM to predict the stock price of Tesla. We will use the monthly stock price data of Tesla from 2015 to 2024 to train the model and then predict the stock price.

## LSTM architecture

LSTM (Long Short-Term Memory) is a type of RNN (Recurrent Neural Network) architecture that is designed to overcome the vanishing gradient problem in traditional RNNs. LSTM has a memory cell that can maintain information over long periods of time, which makes it suitable for time series prediction tasks.
Each cell in an LSTM network has three gates: input gate, forget gate, "gate gate", and output gate. The input gate controls the flow of information into the cell, the forget gate controls the flow of information out of the cell, and the output gate controls the output of the cell. The equations for the gates are as follows:

- Input gate: $i_t = \sigma(W_i \cdot [h_{t-1}, x_t] + b_i)$  
- Forget gate: $f_t = \sigma(W_f \cdot [h_{t-1}, x_t] + b_f)$
- Output gate: $o_t = \sigma(W_o \cdot [h_{t-1}, x_t] + b_o)$
- Gate gate: $g_t = \tanh(W_g \cdot [h_{t-1}, x_t] + b_g)$

The cell state is updated using the following equation:

Cell state: $c_t = f_t \cdot c_{t-1} + i_t \cdot g_t$

The hidden state is updated using the following equation:

Hidden state: $h_t = o_t \cdot \tanh(c_t)$

Equations matrix form:

\begin{equation}
\begin{bmatrix}
i_t \\
f_t \\
o_t \\
g_t
\end{bmatrix}
=
\begin{bmatrix}
\sigma \\
\sigma \\
\sigma \\
\tanh
\end{bmatrix}
\begin{pmatrix}
W_i & W_f & W_o & W_g
\end{pmatrix}
\begin{bmatrix}
h_{t-1} \\
x_t
\end{bmatrix}
+ 
\begin{bmatrix}
b_i \\
b_f \\
b_o \\
b_g
\end{bmatrix}
\end{equation}

We'll start by importing the necessary libraries and loading the data.

In [17]:
import pandas as pd
import plotly.graph_objects as go
import torch
import torch.nn as nn
from torch.nn import MSELoss
from torch.optim.adam import Adam
from torch.utils.data import Dataset, DataLoader

Load the data and preprocess it.

In [18]:
class StockPriceDataset(Dataset):
    def __init__(self, stock_df: pd.DataFrame, seq_length: int, transform=None, target_transform=None) -> None:
        self.stock_df = stock_df
        self.seq_length = seq_length
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self) -> int:
        return len(self.stock_df) - self.seq_length

    def __getitem__(self, idx: int) -> tuple[torch.Tensor, torch.Tensor]:
        x = self.stock_df.iloc[idx:idx + self.seq_length].values
        y = self.stock_df.iloc[idx + self.seq_length].values

        if self.transform:
            x = self.transform(x)

        if self.target_transform:
            y = self.target_transform(y)

        return x, y

In [19]:
stock_df: pd.DataFrame = pd.read_csv(
    'C:\\Users\\stefa\\Documents\\ML\\neural_networks\\data\\tesla_stock_price\\tesla_stock_price.csv')
stock_df = stock_df.dropna()
stock_df = stock_df.iloc[:, 1:]
stock_df['Date'] = pd.to_datetime(stock_df['Date'])
stock_df.set_index('Date', inplace=True)

threshold = int(0.8 * len(stock_df))

train_df: pd.DataFrame = stock_df[:threshold]
test_df: pd.DataFrame = stock_df[threshold:]

# normalize each column
train_mean, test_mean = train_df.mean(), test_df.mean()
train_std, test_std = train_df.std(), test_df.std()

train_df = (train_df - train_mean) / train_std
test_df = (test_df - test_mean) / test_std

train_dataset: Dataset = StockPriceDataset(train_df, seq_length=10, transform=torch.tensor,
                                           target_transform=torch.tensor)
test_dataset: Dataset = StockPriceDataset(test_df, seq_length=10, transform=torch.tensor, target_transform=torch.tensor)

train_loader: DataLoader = DataLoader(train_dataset, batch_size=10)
test_loader: DataLoader = DataLoader(test_dataset, batch_size=10)

fig = go.Figure(data=[go.Candlestick(x=stock_df.index,
                                     open=stock_df['Open'],
                                     high=stock_df['High'],
                                     low=stock_df['Low'],
                                     close=stock_df['Close'])])
fig.show()

In [20]:
class StockPriceLSTM(nn.Module):
    def __init__(self) -> None:
        super(StockPriceLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=5, hidden_size=128, num_layers=2, batch_first=True)
        self.fc = nn.Linear(128, 5)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out


platform = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = StockPriceLSTM().to(platform)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model

StockPriceLSTM(
  (lstm): LSTM(5, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=5, bias=True)
)

In [21]:
def train(model: StockPriceLSTM, train_loader: DataLoader, loss_fn: MSELoss, optimizer: Adam, epochs: int) -> None:
    epochs_no_improve: int = 0
    best_loss: float = float('inf')
    size: int = len(train_loader)
    loss = None

    for epoch in range(epochs):
        model.train()

        for batch, (x, y) in enumerate(train_loader):
            x, y = x.to(platform), y.to(platform)
            optimizer.zero_grad()
            y_pred = model(x.float())
            loss = loss_fn(y_pred, y.float())
            loss.backward()
            optimizer.step()

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')

        test(model, test_loader, loss_fn)

        if loss.item() < best_loss:
            best_loss = loss.item()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve == 5:
            break


def test(model: StockPriceLSTM, test_loader: DataLoader, loss_fn: MSELoss) -> None:
    model.eval()
    test_loss: float = 0
    size = len(test_loader)

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(platform), y.to(platform)
            y_pred: torch.Tensor = model(x.float())
            test_loss += loss_fn(y_pred, y.float()).item()

    test_loss /= size
    print(f'Test average loss: {test_loss:.4f}\n')


train(model, train_loader, loss_fn, optimizer, epochs=50)

Epoch [1/50], Loss: 0.5478
Test average loss: 4.2312

Epoch [2/50], Loss: 0.3295
Test average loss: 1.7987

Epoch [3/50], Loss: 0.3172
Test average loss: 0.7954

Epoch [4/50], Loss: 0.3245
Test average loss: 0.4572

Epoch [5/50], Loss: 0.2659
Test average loss: 0.6277

Epoch [6/50], Loss: 0.1901
Test average loss: 0.4023

Epoch [7/50], Loss: 0.2149
Test average loss: 0.4822

Epoch [8/50], Loss: 0.2669
Test average loss: 0.4418

Epoch [9/50], Loss: 0.1640
Test average loss: 0.3099

Epoch [10/50], Loss: 0.1803
Test average loss: 0.2964

Epoch [11/50], Loss: 0.2557
Test average loss: 0.3268

Epoch [12/50], Loss: 0.1554
Test average loss: 0.2615

Epoch [13/50], Loss: 0.1700
Test average loss: 0.2643

Epoch [14/50], Loss: 0.1907
Test average loss: 0.2605

Epoch [15/50], Loss: 0.1260
Test average loss: 0.2308

Epoch [16/50], Loss: 0.1589
Test average loss: 0.2800

Epoch [17/50], Loss: 0.0900
Test average loss: 0.2057

Epoch [18/50], Loss: 0.1289
Test average loss: 0.2476

Epoch [19/50], Loss

Plot the predicted stock price.

In [22]:
def predict(model: StockPriceLSTM, test_loader: DataLoader) -> list[torch.Tensor]:
    model.eval()
    predictions: list[torch.Tensor] = []

    with torch.no_grad():
        for x, y in test_loader:
            x = x.to(platform)
            y_pred = model(x.float())
            predictions.append(y_pred)

    return predictions


predictions = predict(model, test_loader)
predictions = torch.cat(predictions).cpu().numpy()

# denormalize the data
for i in range(len(predictions)):
    predictions[i] = predictions[i] * test_std.values + test_mean.values

# denormalize test data
test_df = test_df * test_std + test_mean

fig = go.Figure(data=[go.Candlestick(x=test_df.index,
                                     open=test_df['Open'],
                                     high=test_df['High'],
                                     low=test_df['Low'],
                                     close=test_df['Close'],
                                     name='Actual'
                                     )])

fig.add_trace(go.Candlestick(x=test_df.index,
                             open=predictions[:, 0],
                             high=predictions[:, 1],
                             low=predictions[:, 2],
                             close=predictions[:, 3],
                             name='Predicted',
                             increasing_line_color='cyan',
                             decreasing_line_color='gray'
                             ))

fig.show()